In [1]:
import pandas as pd
import os 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
%pwd


'/home/leksman/Desktop/my git hub work/end_to_end_A_I_H_M_S/notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd


'/home/leksman/Desktop/my git hub work/end_to_end_A_I_H_M_S'

In [5]:
df = pd.read_csv("raw_data/brigde_dataset/bridge_data.csv")

In [6]:
df

,Bridge_ID,Location,Age (years),Material,Length (m),Width (m),Height (m),Traffic_Volume (vehicles/day),Weather_Conditions,Water_Flow_Rate (m³/s),...,Stress (MPa),Strain (%),Tensile_Strength (MPa),Rainfall (mm),Material_Composition,Bridge_Design,Construction_Quality,Temperature (°C),Humidity (%),Collapse_Status
0,1,Region_1,26,Steel,50.000000,5.000000,10.000000,100.000000,Rainy,0.00000,...,0.000000,0.000,200.000000,0.000000,"Steel 70%, Concrete 30%",Arch,Good,-30.000000,0.000000,Standing
1,2,Region_2,47,Concrete,50.195020,5.004500,10.009001,100.990099,Sunny,0.50005,...,0.010001,0.001,200.080008,0.050005,"Concrete 80%, Wood 20%",Beam,Good,-29.991999,0.010001,Standing
2,3,Region_3,1,Wood,50.390039,5.009001,10.018002,101.980198,Snowy,1.00010,...,0.020002,0.002,200.160016,0.100010,"Concrete 80%, Wood 20%",Truss,Good,-29.983998,0.020002,Standing
3,4,Region_4,32,Steel,50.585059,5.013501,10.027003,102.970297,Cloudy,1.50015,...,0.030003,0.003,200.240024,0.150015,"Steel 70%, Concrete 30%",Arch,Good,-29.975998,0.030003,Standing
4,5,Region_5,27,Concrete,50.780078,5.018002,10.036004,103.960396,Windy,2.00020,...,0.040004,0.004,200.320032,0.200020,"Concrete 80%, Wood 20%",Beam,Good,-29.967997,0.040004,Standing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,Region_1,13,Wood,1999.219922,49.981998,99.963996,9996.039604,Rainy,4997.99980,...,99.959996,9.996,999.679968,499.799980,"Concrete 80%, Wood 20%",Truss,Good,49.967997,99.959996,Collapsed
9996,9997,Region_2,6,Steel,1999.414941,49.986499,99.972997,9997.029703,Sunny,4998.49985,...,99.969997,9.997,999.759976,499.849985,"Steel 70%, Concrete 30%",Arch,Good,49.975998,99.969997,Collapsed
9997,9998,Region_3,44,Concrete,1999.609961,49.990999,99.981998,9998.019802,Snowy,4998.99990,...,99.979998,9.998,999.839984,499.899990,"Concrete 80%, Wood 20%",Beam,Good,49.983998,99.979998,Collapsed
9998,9999,Region_4,5,Wood,1999.804980,49.995500,99.990999,9999.009901,Cloudy,4999.49995,...,99.989999,9.999,999.919992,499.949995,"Concrete 80%, Wood 20%",Truss,Good,49.991999,99.989999,Collapsed


In [7]:
df = df.drop(["Location","Bridge_ID"],axis=1) 

In [8]:
df

,Age (years),Material,Length (m),Width (m),Height (m),Traffic_Volume (vehicles/day),Weather_Conditions,Water_Flow_Rate (m³/s),Maintenance_History,Stress (MPa),Strain (%),Tensile_Strength (MPa),Rainfall (mm),Material_Composition,Bridge_Design,Construction_Quality,Temperature (°C),Humidity (%),Collapse_Status
0,26,Steel,50.000000,5.000000,10.000000,100.000000,Rainy,0.00000,2010-01-01,0.000000,0.000,200.000000,0.000000,"Steel 70%, Concrete 30%",Arch,Good,-30.000000,0.000000,Standing
1,47,Concrete,50.195020,5.004500,10.009001,100.990099,Sunny,0.50005,2010-01-02,0.010001,0.001,200.080008,0.050005,"Concrete 80%, Wood 20%",Beam,Good,-29.991999,0.010001,Standing
2,1,Wood,50.390039,5.009001,10.018002,101.980198,Snowy,1.00010,2010-01-03,0.020002,0.002,200.160016,0.100010,"Concrete 80%, Wood 20%",Truss,Good,-29.983998,0.020002,Standing
3,32,Steel,50.585059,5.013501,10.027003,102.970297,Cloudy,1.50015,2010-01-04,0.030003,0.003,200.240024,0.150015,"Steel 70%, Concrete 30%",Arch,Good,-29.975998,0.030003,Standing
4,27,Concrete,50.780078,5.018002,10.036004,103.960396,Windy,2.00020,2010-01-05,0.040004,0.004,200.320032,0.200020,"Concrete 80%, Wood 20%",Beam,Good,-29.967997,0.040004,Standing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13,Wood,1999.219922,49.981998,99.963996,9996.039604,Rainy,4997.99980,2037-05-14,99.959996,9.996,999.679968,499.799980,"Concrete 80%, Wood 20%",Truss,Good,49.967997,99.959996,Collapsed
9996,6,Steel,1999.414941,49.986499,99.972997,9997.029703,Sunny,4998.49985,2037-05-15,99.969997,9.997,999.759976,499.849985,"Steel 70%, Concrete 30%",Arch,Good,49.975998,99.969997,Collapsed
9997,44,Concrete,1999.609961,49.990999,99.981998,9998.019802,Snowy,4998.99990,2037-05-16,99.979998,9.998,999.839984,499.899990,"Concrete 80%, Wood 20%",Beam,Good,49.983998,99.979998,Collapsed
9998,5,Wood,1999.804980,49.995500,99.990999,9999.009901,Cloudy,4999.49995,2037-05-17,99.989999,9.999,999.919992,499.949995,"Concrete 80%, Wood 20%",Truss,Good,49.991999,99.989999,Collapsed


In [9]:
cat_df = df.select_dtypes(include="object")

In [10]:
columns = cat_df.columns.to_list()

In [11]:
for feature in columns:
    
    print(f"{feature} have  {len(df[feature].unique())}  values , they are:{df[feature].unique()}")

Material have  3  values , they are:['Steel' 'Concrete' 'Wood']
Weather_Conditions have  5  values , they are:['Rainy' 'Sunny' 'Snowy' 'Cloudy' 'Windy']
Maintenance_History have  10000  values , they are:['2010-01-01' '2010-01-02' '2010-01-03' ... '2037-05-16' '2037-05-17'
 '2037-05-18']
Material_Composition have  2  values , they are:['Steel 70%, Concrete 30%' 'Concrete 80%, Wood 20%']
Bridge_Design have  3  values , they are:['Arch' 'Beam' 'Truss']
Construction_Quality have  1  values , they are:['Good']
Collapse_Status have  2  values , they are:['Standing' 'Collapsed']


### Below are short text categories that is why i use labelEncoder

In [12]:
le = LabelEncoder()
df['Material'] = le.fit_transform(df['Material'])
df['Bridge_Design'] = le.fit_transform(df['Bridge_Design'])

### I use one-hot-encoder for whether_condiction because the unique values ain't in order because each vaule have there own important

In [13]:
df = pd.get_dummies(df, columns=['Weather_Conditions'], drop_first=True,dtype=int)

### I create new columns for Maintenance data History because i want the model to learn time based trend from the dataset

In [14]:
df['Maintenance_History'] = pd.to_datetime(df['Maintenance_History'])
df['Maintenance_Year'] = df['Maintenance_History'].dt.year
df['Maintenance_Month'] = df['Maintenance_History'].dt.month

# drop the original column
df.drop('Maintenance_History', axis=1, inplace=True)


### I reformated ['Steel 70%, Concrete 30%' 'Concrete 80%, Wood 20%'] to new columns 

In [15]:
df['Steel_%'] = df['Material_Composition'].apply(lambda x: int(x.split('Steel ')[1].split('%')[0]) if 'Steel' in x else 0)
df['Concrete_%'] = df['Material_Composition'].apply(lambda x: int(x.split('Concrete ')[1].split('%')[0]) if 'Concrete' in x else 0)
df['Wood_%'] = df['Material_Composition'].apply(lambda x: int(x.split('Wood ')[1].split('%')[0]) if 'Wood' in x else 0)

df.drop('Material_Composition', axis=1, inplace=True)


### I droped Construction_Quality because it only has a unique value

In [16]:
df.drop('Construction_Quality', axis=1, inplace=True)


### Converted my target label into 0s and 1s

In [17]:
df['Collapse_Status'] = df['Collapse_Status'].map({'Standing': 0, 'Collapsed': 1})

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Age (years)                    10000 non-null  int64  
 1   Material                       10000 non-null  int64  
 2   Length (m)                     10000 non-null  float64
 3   Width (m)                      10000 non-null  float64
 4   Height (m)                     10000 non-null  float64
 5   Traffic_Volume (vehicles/day)  10000 non-null  float64
 6   Water_Flow_Rate (m³/s)         10000 non-null  float64
 7   Stress (MPa)                   10000 non-null  float64
 8   Strain (%)                     10000 non-null  float64
 9   Tensile_Strength (MPa)         10000 non-null  float64
 10  Rainfall (mm)                  10000 non-null  float64
 11  Bridge_Design                  10000 non-null  int64  
 12  Temperature (°C)               10000 non-null  

In [19]:
df.isnull().sum()

Age (years)                      0
Material                         0
Length (m)                       0
Width (m)                        0
Height (m)                       0
Traffic_Volume (vehicles/day)    0
Water_Flow_Rate (m³/s)           0
Stress (MPa)                     0
Strain (%)                       0
Tensile_Strength (MPa)           0
Rainfall (mm)                    0
Bridge_Design                    0
Temperature (°C)                 0
Humidity (%)                     0
Collapse_Status                  0
Weather_Conditions_Rainy         0
Weather_Conditions_Snowy         0
Weather_Conditions_Sunny         0
Weather_Conditions_Windy         0
Maintenance_Year                 0
Maintenance_Month                0
Steel_%                          0
Concrete_%                       0
Wood_%                           0
dtype: int64

In [20]:
df.describe().corr()["Collapse_Status"]

Age (years)                      0.999991
Material                         1.000000
Length (m)                       0.982584
Width (m)                        0.999992
Height (m)                       0.999967
Traffic_Volume (vehicles/day)    0.520125
Water_Flow_Rate (m³/s)           0.869189
Stress (MPa)                     0.999962
Strain (%)                       1.000000
Tensile_Strength (MPa)           0.996716
Rainfall (mm)                    0.999011
Bridge_Design                    1.000000
Temperature (°C)                 0.999977
Humidity (%)                     0.999962
Collapse_Status                  1.000000
Weather_Conditions_Rainy         1.000000
Weather_Conditions_Snowy         1.000000
Weather_Conditions_Sunny         1.000000
Weather_Conditions_Windy         1.000000
Maintenance_Year                 0.972082
Maintenance_Month                1.000000
Steel_%                          0.999966
Concrete_%                       0.999976
Wood_%                           0

In [21]:
X = df.drop("Collapse_Status",axis=1)
y = df["Collapse_Status"]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
scalter_model = StandardScaler()
scaled_x_train = scalter_model.fit_transform(X_train)
scaled_x_test = scalter_model.transform(X_test)